<a href="https://colab.research.google.com/github/Falconwatch/SberUni-ChooseMLModel/blob/main/AutoML_Sber_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В рамках домашнего задания мы предлагаем вам поучаствовать в соревновании [Kaggle](https://www.kaggle.com/t/7e9b43b6978b4c18a58c6b60a2da2ed8). В этом соревновании вы будете прогнозировать цены на автомобили с помощью фрэймворка для AutoML [LightAutoML](https://github.com/sberbank-ai-lab/LightAutoML) by Sberbank AI Lab.


Сначала вы получите базовое решение, а затем сможете посоревноваться между собой и другими участниками на платформе Kaggle.






# Установка необходимых пакетов и загрузка данных

In [ ]:
# Google Colab has some problem with install lightautoml
# So, we need to straightforward highlight the last version of packge
!pip install transformers # additional lib for lightautonl (optional)
!pip install lightautoml==0.3.8b1 # lightautoml

# If you work on your own computer, you can use
#!pip install -U lightautoml

# additional
!pip install multiprocessing

!pip install -q vininfo

!pip install gdown # for data loading
!gdown 1O7n3ztzKWpY6x2vha7uuKBqksE1LjhEn # load training data
!gdown 1gvTvn71-K6mIm_hvBWl9XKKtcqNuoRhp # load test data
!gdown 1LyayILm9dUzifg55h3w1eAHkizGMOgxZ # load sample submission

from IPython.display import clear_output
clear_output()

# Импортирование необходимых библиотек

In [ ]:
import multiprocessing as mp
import pandas as pd

from sklearn.metrics import mean_absolute_error
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import numpy as np
import matplotlib.pyplot as plt

# Загрузка данных

Данные соревнования представляют собой таблицы с некоторым набором характеристик машин и целевой переменной - цена продажи.

Описание признаков:
* row_id - ID машины
* vehicle_manufacturer - производитель машины
* vehicle_model - модель машины
* vehicle_category - тип кузова
* current_mileage - текущий пробег
* vehicle_year - год выпуска
* vehicle_gearbox_type - тип коробки передач
* doors_cnt - кол-во дверей
* wheels - тип машины (праворульная/леворульная)
* vehicle_color - цвет машины
* vehicle_interior_color - цвет салона
* car_vin - идентификационный номер машины
* car_leather_interior - идентификатор, кожаный ли салон
* deal_type - машина продается или арендуется

**Целевая переменная** - final_price

Загрузим обучающую и тестовую выборки и посмотрим на них.

In [ ]:
train_data = pd.read_csv('./train_data.csv')
train_data.head()

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type,final_price
0,0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,Right-hand drive,Silver,Black,NaN,0,For Sale,3650.0
1,1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale,6800.0
2,2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,Left wheel,Silver,Black,KMHTC6AE3GU293912,1,For Sale,6300.0
3,3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,Left wheel,Blue,Black,NaN,1,For Sale,14488.0
4,4,TOYOTA,CHR,Jeep,18950,2019,Automatic,4/5,Left wheel,Black,NaN,JTNKHMBX7K1030253,1,For Sale,5000.0


In [ ]:
test_data = pd.read_csv('./test_data.csv')
test_data.head()

,row_ID,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_vin,car_leather_interior,deal_type
0,35000,TOYOTA,Prius,Hatchback,323733,2012,Automatic,4/5,Left wheel,Grey,Black,JTDKN3DU6C5439638,1,For Sale
1,35001,HYUNDAI,Elantra,Sedan,112000,2013,Tiptronic,4/5,Left wheel,Grey,Black,SURATSHIA,1,For Sale
2,35002,LEXUS,NX 300,Jeep,16920,2018,Automatic,NaN,Left wheel,Brown,NaN,JTJYARBZ5J2104521,1,For Sale
3,35003,LEXUS,CT 200h,Hatchback,302742,2012,Automatic,4/5,Left wheel,White,NaN,JTHKD5BH4C2070945,1,For Sale
4,35004,TOYOTA,RAV 4,Jeep,1800,2002,Manual,4/5,Left wheel,Silver,Black,NaN,0,For Sale


# FE

## Чистка

In [ ]:
def dataset_strip_strings(data):
    """Удаление лишних пробелов в строковых полях набора данных"""
    for field in data.select_dtypes(include='object'):
        data[field] = data[field].str.strip()
        data[field] = data[field].str.upper()
    return data

train_data = dataset_strip_strings(train_data)
test_data = dataset_strip_strings(test_data)

## Возраст авто

In [ ]:
def auto_age(dataset):
  dataset["auto_age"] = dataset["vehicle_year"].max() - dataset["vehicle_year"]

auto_age(train_data)
auto_age(test_data)

## Полное название машины

In [ ]:
def full_name(dataset):
  dataset['full_name'] = dataset['vehicle_manufacturer'] + dataset['vehicle_model'].fillna('')

full_name(train_data)
full_name(test_data)

## Информация из VIN

In [ ]:
from vininfo import Vin
def vin_to_dic(vin_no):
    """Преобразует VIN-номер в словарь параметров автомобиля"""
    vin_dic = {}

    try:
        vin_info = Vin(vin_no)

        vin_dic['checksum_is_ok'] = vin_info.verify_checksum()
        vin_dic['country'] = vin_info.country
        vin_dic['manufacturer'] = vin_info.manufacturer
        vin_dic['region'] = vin_info.region
        vin_dic['produce_year'] = vin_info.years[0]
        vin_dic['model_year'] = vin_info.years[1]
        vin_dic['wmi'] = vin_info.wmi      # всемирный индекс изготовителя
        vin_dic['vds'] = vin_info.vds[:-1] # технические характеристики автомобиля
        vin_dic['vis'] = vin_info.vis      # идентификационный номер автомобиля

        details = vin_info.details
        if details:
            vin_dic['details'] = True
            vin_dic['body'] = str(details.body)
            vin_dic['engine'] = str(details.engine)
            vin_dic['model'] = str(details.model)
            vin_dic['plant'] = str(details.plant)
            vin_dic['serial'] = str(details.serial)
            vin_dic['transmission'] = str(details.transmission)
        else:
            vin_dic['details'] = False

            for field in ['body', 'engine', 'model', 'plant', 'serial', 'transmission']:
                vin_dic[field] = None
    except:
        vin_dic['checksum_is_ok'] = False

    return vin_dic

In [ ]:
vin_train_data = pd.DataFrame(list(train_data["car_vin"].apply(vin_to_dic)))
vin_test_data = pd.DataFrame(list(test_data["car_vin"].apply(vin_to_dic)))

In [ ]:
if "wmi" not in train_data.columns:
  train_data = pd.concat([train_data,
                          vin_train_data[["checksum_is_ok", "country", "manufacturer", "region", "produce_year", "model_year", "wmi"]].copy()],
                          axis=1)

if "wmi" not in test_data.columns:
  test_data = pd.concat([test_data,
                          vin_test_data[["checksum_is_ok", "country", "manufacturer", "region", "produce_year", "model_year", "wmi"]].copy()],
                          axis=1)

## Редкие цвета

In [ ]:
car_colors = train_data["vehicle_color"].value_counts()
dict_extra_vehicle_color = {k:1 for k in car_colors[car_colors<=1000].index.values}

In [ ]:
train_data["has_rare_color"] = train_data["vehicle_color"].map(dict_extra_vehicle_color)
train_data["has_rare_color"] .fillna(0, inplace=True)
test_data["has_rare_color"] = test_data["vehicle_color"].map(dict_extra_vehicle_color)
test_data["has_rare_color"] .fillna(0, inplace=True)

## Средний пробег

In [ ]:
train_data["millage_yearly"] = train_data["current_mileage"] / train_data["auto_age"]

## Доп фичи

In [ ]:
def create_extra_features(data):
    data['NANs_cnt'] = data.isnull().sum(axis = 1)

def create_col_with_min_freq(data, col, min_freq = 10):
    # replace rare values (less than min_freq rows) in feature by RARE_VALUE
    data[col + '_fixed'] = data[col].astype(str)
    data.loc[data[col + '_fixed'].value_counts()[data[col + '_fixed']].values < min_freq, col + '_fixed'] = "RARE_VALUE"
    data.replace({'nan': np.nan}, inplace = True)

def create_gr_feats(data):
    # create aggregation feats for numeric features based on categorical ones
    for cat_col in ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category',
                   'vehicle_gearbox_type', 'doors_cnt', 'wheels', 'vehicle_color',
                   'vehicle_interior_color', 'deal_type']:
        create_col_with_min_freq(data, cat_col, 15)
        for num_col in ['current_mileage', 'vehicle_year', 'car_leather_interior']:
            for n, f in [('mean', np.mean), ('min', np.nanmin), ('max', np.nanmax)]:
                data['FIXED_' + n + '_' + num_col + '_by_' + cat_col] = data.groupby(cat_col + '_fixed')[num_col].transform(f)

    # create features with counts
    for col in ['vehicle_manufacturer', 'vehicle_model', 'vehicle_category',
               'current_mileage', 'vehicle_year', 'vehicle_gearbox_type', 'doors_cnt',
               'wheels', 'vehicle_color', 'vehicle_interior_color', 'car_vin', 'deal_type']:
        data[col + '_cnt'] = data[col].map(data[col].value_counts(dropna = False))



create_extra_features(train_data)
create_extra_features(test_data)

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]
print(train_data.shape, test_data.shape)

(35000, 129) (10697, 129)


## Число перепродаж

In [ ]:
sale_train_data = train_data[train_data["deal_type"]=="FOR SALE"].copy(deep=True)
sale_test_data = test_data[test_data["deal_type"]=="FOR SALE"].copy(deep=True)

In [ ]:
sale_train_data['vin_cum_count'] = 0
sale_test_data['vin_cum_count'] = 0

vin_cumcount_train = sale_train_data[~sale_train_data.car_vin.isna()].groupby('car_vin' )['car_vin'].cumcount() + 1
sale_train_data.loc[vin_cumcount_train.index, 'vin_cum_count'] = vin_cumcount_train

vin_cumcount_test = sale_test_data[~sale_test_data.car_vin.isna()].groupby('car_vin' )['car_vin'].cumcount() + 1
sale_test_data.loc[vin_cumcount_test.index, 'vin_cum_count'] = vin_cumcount_test

In [ ]:
train_data = train_data.merge(sale_train_data[["row_ID", "vin_cum_count"]], on = "row_ID", how="left")
train_data["vin_cum_count"].fillna(-1, inplace=True)

test_data = test_data.merge(sale_test_data[["row_ID", "vin_cum_count"]], on = "row_ID", how="left")
test_data["vin_cum_count"].fillna(-1, inplace=True)

# Запуск AutoML

In [ ]:
N_THREADS = mp.cpu_count()
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2 # Test size for metric check
time_out_mins = 60
TIMEOUT = 60 * time_out_mins
TARGET_NAME = 'final_price' # Target column name

In [ ]:
nubiques = train_data.nunique().sort_values()#[:20]

In [ ]:
nubiques[nubiques<200].index.values

array(['FIXED_max_car_leather_interior_by_vehicle_manufacturer',
       'FIXED_min_car_leather_interior_by_doors_cnt',
       'FIXED_min_current_mileage_by_wheels',
       'FIXED_min_car_leather_interior_by_vehicle_interior_color',
       'FIXED_max_car_leather_interior_by_vehicle_interior_color',
       'FIXED_min_current_mileage_by_deal_type',
       'FIXED_max_vehicle_year_by_doors_cnt',
       'FIXED_min_car_leather_interior_by_vehicle_category',
       'FIXED_max_current_mileage_by_deal_type',
       'FIXED_min_current_mileage_by_vehicle_gearbox_type',
       'FIXED_max_vehicle_year_by_deal_type',
       'FIXED_min_car_leather_interior_by_wheels',
       'FIXED_min_car_leather_interior_by_deal_type',
       'FIXED_max_car_leather_interior_by_deal_type',
       'FIXED_min_current_mileage_by_vehicle_color',
       'FIXED_max_car_leather_interior_by_doors_cnt',
       'FIXED_max_current_mileage_by_doors_cnt',
       'FIXED_min_current_mileage_by_vehicle_interior_color',
       'FIXED

In [ ]:
task = Task('reg', loss='mae', metric = 'mae')

roles = {
        'target': TARGET_NAME,
        'id': ["row_ID"],
        'category': nubiques[nubiques<20].index.values
    }


In [ ]:
automl = TabularUtilizedAutoML(task = task,
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(train_data, roles = roles)
#logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

INFO:lightautoml.addons.utilization.utilization:Start automl utilizator with listed constraints:
INFO:lightautoml.addons.utilization.utilization:- time: 3600.00 seconds
INFO:lightautoml.addons.utilization.utilization:- CPU: 2 cores
INFO:lightautoml.addons.utilization.utilization:- memory: 16 GB

INFO:lightautoml.addons.utilization.utilization:If one preset completes earlier, next preset configuration will be started

INFO:lightautoml.addons.utilization.utilization:==================================================
INFO:lightautoml.addons.utilization.utilization:Start 0 automl preset configuration:
INFO:lightautoml.addons.utilization.utilization:conf_0_sel_type_0.yml, random state: {'reader_params': {'random_state': 42}, 'nn_params': {'random_state': 42}, 'general_params': {'return_all_predictions': False}}
INFO3:lightautoml.addons.utilization.utilization:Found reader_params in kwargs, need to combine
INFO3:lightautoml.addons.utilization.utilization:Merged variant for reader_params = {'

In [ ]:
oof_pred.shape

(35000, 1)

## Предсказания на тесте и качество на трейне

In [ ]:
test_pred = automl.predict(test_data)
mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])

2198.1261145032813

# Предсказание

In [ ]:
submission = pd.read_csv('sample_submission.csv')
print(submission.shape)
submission.head()

(10697, 2)


,row_ID,final_price
0,35000,0
1,35001,0
2,35002,0
3,35003,0
4,35004,0


In [ ]:
test_pred.data[:, 0]

array([ 2826.092 ,  5610.201 ,  2437.9712, ..., 17480.807 ,  5128.2896,
        6421.3545], dtype=float32)

In [ ]:
sub = test_data[["row_ID"]].copy()
sub["final_price"] = test_pred.data[:, 0]

## Лик

In [ ]:
a = train_data.loc[train_data.car_vin.str.len() == 19].car_vin.value_counts()
leaks = train_data.loc[train_data.car_vin.isin(list(a[a == 1].index))]
full_df = pd.merge(leaks[['car_vin', 'final_price', 'deal_type', 'vehicle_category']],
                   test_data.loc[test_data.car_vin.str.len() == 19, ['row_ID', 'car_vin', 'deal_type', 'vehicle_category']],
                   how='inner',
                   on=['car_vin', 'deal_type', 'vehicle_category'])
for row in list(full_df.row_ID.values):
    sub.loc[sub.row_ID == row, 'final_price'] = full_df.loc[full_df.row_ID == row, 'final_price'].values

## Сохраняем

In [ ]:
from datetime import datetime
sub.to_csv('shcherbakov_ML_model_selection_HW2_{}.csv'.format(str(datetime.now())),
                        index = False)

Полученное решение можно загрузить на соревнование [Kaggle](https://www.kaggle.com/t/7e9b43b6978b4c18a58c6b60a2da2ed8).


**В качестве домашнего задания мы предлагаем вам самостоятельно поэксперементировать с LightAutoML, попробовать побить текущее решение и посоревноваться между собой.**

# Что можно ещё попробовать?

* Как бы ни был хорош AutoML, data scientist обладает несомненным преимуществом в виде своего опыта:) Например, вы можете придумать сложные признаки и/или ручной препроцессинг данных перед тем, как запускать LightAutoML. Какие идеи могут быть полезны:  
  * убрать редкие значения (outliers);
  * посчитать различные аггрегированные значения, например, средние значения числовых признаков по разным группам или частоту возникновения категорий;   
  * поработать с VIN номером машины (см. [VIN](https://ru.wikipedia.org/wiki/%D0%98%D0%B4%D0%B5%D0%BD%D1%82%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BD%D0%BE%D0%BC%D0%B5%D1%80_%D1%82%D1%80%D0%B0%D0%BD%D1%81%D0%BF%D0%BE%D1%80%D1%82%D0%BD%D0%BE%D0%B3%D0%BE_%D1%81%D1%80%D0%B5%D0%B4%D1%81%D1%82%D0%B2%D0%B0))  
  * выделить "особые фишки" машины (например, необычный цвет);
  * выделить дополнительные группы в данных, например, объединить производителей по странам и/или модели по "премиальности" итд.

* Рассмотреть другой набор моделей.
* Попробовать мягче ограничивать перебор решений (увеличить TIMEOUT и/или поменять TabularAutoML на TabularUtilizedAutoML).
* "Поиграться" с ролями.
* Настроить пайплайн [полностью самим](https://github.com/sb-ai-lab/LightAutoML/blob/master/examples/tutorials/Tutorial_6_custom_pipeline.ipynb)) - высший пилотаж :)


# Баллы за эксперименты распределяются следующим образом:
* Реализована какая-нибудь генерация признаков, которая хоть как-то улучшает базовое решение - 5 баллов;
* За особо креативные подходы в генерации признаков - до 10 баллов дополнительно;
* Подобраны гиперпараметры TabularAutoML - до 5 баллов;
* Отдельно оцениваются эксперименты с TIMEOUT (в т.ч. мотивированный переход на TabularUtilizedAutoML) - 3 балла;
------

Особые бонусы:

* Реализован кастомный pipeline AutoML - до 10 баллов;
* Топ K (K $\in$ {1, 2, 3}) на Kaggle - (4 - K) * 5 баллов.


![Good Luck!](https://images.fineartamerica.com/images/artworkimages/mediumlarge/2/no-prob-lama-magdalena-walulik.jpg)
